In [423]:
import graphviz
from datetime import datetime
import json
import numpy as np
import time

In [424]:
names = ['smallRandom','xsmallComplex','smallComplex','mediumRandom','MediumComplex','largeComplex','xlargeComplex','xxlargeComplex']
def create_path(names):
    paths = []
    for name in names : 
        path = './graphs/{}.json'.format(name)
        paths.append(path)
    return paths
paths = create_path(names)
print(paths[0])

./graphs/smallRandom.json


In [425]:
print(paths)
file_num = 1

['./graphs/smallRandom.json', './graphs/xsmallComplex.json', './graphs/smallComplex.json', './graphs/mediumRandom.json', './graphs/MediumComplex.json', './graphs/largeComplex.json', './graphs/xlargeComplex.json', './graphs/xxlargeComplex.json']


In [426]:
with open(paths[file_num], 'r') as f:
    data = json.load(f)

nodes_save = data['nodes']


In [427]:
nodes = nodes_save.copy()

In [428]:
def create_successors():
    ready_to_start = {}
    for node in nodes :
        nodes[node]['Successors'] = []
    for node in nodes :
        depend = nodes[node]['Dependencies']
        if len(depend)>0:
            for i in depend :
                nodes[str(i)]['Successors'].append(node)
        else :
            ready_to_start[node]=0
            # print(node)
    return ready_to_start

In [429]:
# create_successors()
# nodes

In [430]:
def weight(node):
    pt = node['Data'].split(':')
    time_s = float(pt[0])*3600 + float(pt[1])*60 + float(pt[2])
    return time_s

In [431]:
def add_ranks1():
    for node in nodes:
        nodes[node]['Weight'] = weight(nodes[node])
        nodes[node]['rank'] = 0
        nodes[node]['rank_calculated'] = False
        nodes[node]['finished'] = -1
def add_ranks2():
    for node in nodes:
        nodes[node]['Weight'] = weight(nodes[node])
        nodes[node]['finished'] = False

# for node in nodes.values() :
#     print(node['rank'])
#     print(node['rank_calculated'])

In [432]:
def rec_calc_rank1(nodes, node):
    succ = nodes[node]['Successors']
    if len(succ) == 0 or 'rank' in nodes[node]:
        nodes[node]['rank'] = nodes[node]['Weight']
        return nodes[node]['Weight']
    else:
        m = 0
        for i in succ:
            u = rec_calc_rank1(nodes, i)
            if m < u :
                m = u
        nodes[node]['rank'] = nodes[node]['Weight'] + m
        return nodes[node]['rank']



In [433]:
def rec_calc_rank2(nodes, traversal):
    stop = False
    traversal_copy = traversal.copy()
    node = traversal_copy[-1]

    succ = nodes[node]['Successors']
    if len(succ) == 0 :
        nodes[node]['rank'] = nodes[node]['Weight']
        traversal_copy.pop()
        return traversal_copy
    m = 0
    for i in succ :
        if 'rank' in nodes[i] :
            u = nodes[i]['rank']
            if m < u :
                m = u
        else :
            traversal_copy.append(i)
            stop = True
    if not stop :
        nodes[node]['rank'] = nodes[node]['Weight'] + m
        traversal_copy.pop()
    return traversal_copy



In [434]:
def calculate_rank(node):
    if not nodes[node]['rank_calculated']:
        succ = nodes[node]['Successors']     
        rank = nodes[node]['Weight']
        if len(succ) > 0:
            for i in succ:
                if not nodes[str(i)]['rank_calculated']:
                    # print("couldn't calculate rank")
                    return 1  
            weights = [nodes[str(i)]['rank'] for i in succ]
            # print('node weight:',rank)
            # print('successors:',succ)
            # print('succ weights:',weights)
            max_weights = max(weights)
            # print('max:',max_weights)
            rank += max_weights
        nodes[node]['rank'] = rank
        # print('calculated rank:', rank)
        nodes[node]['rank_calculated'] = True
    return 0

In [435]:
def graph_update():
    count = 0
    for node in reversed(nodes.keys()):
        # print('node:',node)
        count += calculate_rank(node)
    # print('not calculated ranks:',count)    
    return count

In [436]:
def calculate_ranks():
    epoch = 1
    count = 1
    while count > 0:
        # print('epoch:',epoch)
        count = graph_update()
        epoch+=1
    # print('rank calculation done')

In [437]:
def make_priority_list():
    node_list = []
    rank_list = []
    for node in nodes:
        node_list.append(node)
        rank_list.append(nodes[node]['rank'])

    # print(node_list)
    # print(rank_list)
    # indexes = np.flip(np.argsort(rank_list))
    # sorted_nodes = [node_list[i] for i in indexes]
    sorted_nodes = [x for _,x in sorted(zip(rank_list,node_list), reverse=True)]
    return sorted_nodes

In [438]:
def process_graph1():
    ready_to_start = create_successors()
    add_ranks1()
    calculate_ranks()
    priority_list = make_priority_list()
    return ready_to_start, priority_list

def process_graph2():
    ready_to_start = create_successors()
    add_ranks2()
    traversal = [node for node in ready_to_start]
    while len(traversal)>0 :
        # print(traversal)
        traversal = rec_calc_rank2(nodes, traversal)
    print(len(nodes))
    count = 0
    for node in nodes :
        if 'rank' not in nodes[node]:
            # print(node)
            count +=1
    print(count)
    print(nodes['1535']['Successors'])
    print(nodes['919']['rank'])
    priority_list = make_priority_list()
    return ready_to_start, priority_list

In [439]:
nodes = nodes_save.copy()
start_time = time.time()
ready_to_start, priority_list = process_graph2()
duration = time.time() - start_time
# print(priority_list)
# print(len(nodes))
# print(len(priority_list))
print('duration:', duration)


1181
0
['1531']
1435770.9733583005
duration: 0.019947052001953125


In [440]:
print(ready_to_start)
print(priority_list)

{'919': 0}
['919', '927', '930', '931', '933', '934', '915', '917', '918', '920', '928', '929', '932', '516', '517', '518', '519', '520', '525', '526', '527', '521', '522', '523', '524', '947', '949', '957', '960', '956', '958', '959', '961', '954', '955', '966', '969', '972', '878', '880', '883', '886', '892', '893', '885', '887', '889', '890', '891', '881', '882', '884', '888', '900', '291', '292', '293', '294', '295', '300', '301', '302', '296', '297', '298', '299', '1111', '1113', '1117', '1123', '1129', '1132', '1115', '1116', '1122', '1126', '1112', '1114', '1118', '1124', '1128', '1131', '1091', '1094', '1096', '1103', '1098', '1099', '1100', '1120', '544', '545', '580', '1127', '546', '581', '1092', '547', '1093', '1097', '548', '570', '582', '583', '552', '571', '553', '588', '589', '572', '590', '573', '591', '554', '577', '584', '555', '578', '579', '556', '585', '574', '586', '557', '575', '549', '576', '550', '551', '587', '1199', '1200', '1206', '1208', '1218', '1220', '1

In [441]:
# times = []
# priority_lists=[]
# for path in paths :
#     print('proccessing graph:', path)   
#     with open(path, 'r') as f:
#         data = json.load(f)
#     nodes = data['nodes']
#     start_time = time.time()
#     priority_list = process_graph()
#     end_time = time.time()
#     print('duration :', end_time - start_time)
#     priority_lists.append(priority_list)
#     core_times = calculate_execution_time(priority_list,2)
#     execution_time = min(core_times)
#     print(execution_time)
#     times.append(execution_time)
#     del(nodes)

# print(times)

    

In [442]:
priority_list_save = priority_list.copy()
ready_to_start_save = ready_to_start.copy()

In [443]:
# import matplotlib.pyplot as plt
# plt.figure()
# plt.title('evolution of graph execution time')
# plt.plot(times)


In [444]:
def make_empty_schedule(n_cores):
    schedule = {i:[] for i in range(n_cores)}
    return schedule

def reinitialise_finished():
    for node in nodes :
        nodes[node]['finished'] = -1
n_cores = 2
schedule = make_empty_schedule(n_cores)
proc_end_times={core:0 for core in range(n_cores)}
print(schedule)

{0: [], 1: []}


In [445]:

def get_end_times(schedule):
    end_times = {}
    cores_that_did_smth = []
    for core in schedule : 
        if len(schedule[core])==0 :
            end_times[core] = 0
        else :
            end_times[core] = schedule[core][-1]['end']
            cores_that_did_smth.append(core)

    return end_times

def update_ready_to_start(finished_task, task_end_time):
    # ready_to_start_copy = ready_to_start.copy()
    nodes[finished_task]['finished'] = task_end_time
    succ = nodes[finished_task]['Successors']
    if len(succ)>0:
        for node in succ:
            dep_done = [nodes[str(dep)]['finished']!=-1 for dep in nodes[node]['Dependencies']]
            if not (False in dep_done) :
                dep_end_times = [nodes[str(dep)]['finished'] for dep in nodes[node]['Dependencies']]
                ready_to_start[node] = max(dep_end_times)

def gef_first_core_ready(end_times):
    end_times_copy = end_times.copy()
    ind = min(end_times_copy, key=end_times.get)
    task_end_time = end_times_copy.pop(ind)
    # if len(cores_that_did_smth)>0 :
    #     if ind in cores_that_did_smth : 
    #         finished_task = schedule[ind][-1]['task']
    #         print('finished task :', finished_task)
    #         update_ready_to_start(finished_task, task_end_time)
    #         print('ready to start', ready_to_start)
    # else :
    #     print('no tasks done')
    return ind, task_end_time, end_times_copy

def get_next_task():
    task = priority_list[0]
    return task

def try_assign_process(task, core, available_time_start, available_time_end):
    # print(nodes[task]['finished'])
    if len(ready_to_start)>0 and (nodes[task]['finished']==-1):
        if task in ready_to_start :
            if available_time_end == -1 or ready_to_start[task] < available_time_end :
                start_time = max(available_time_start, ready_to_start[task])
                end_time = start_time + nodes[task]['Weight']
                schedule[core].append({'task':task, 'start':start_time, 'end':end_time})
                nodes[task]['done'] = True
                # print('assigning task',task)
                update_ready_to_start(task, end_time)
                ready_to_start.pop(task)
                if len(priority_list)>0 :     
                    for i in range(len(priority_list)):
                        if task == priority_list[i] : 
                            priority_list.pop(i)
                            return True
                return True 
        else :
            print('task {} not in ready to start'.format(task))
    else :
        print('ready_to_start is empty')   
    return False

def update_dead_time(dead_time_list, done, time_stamp, ind):
    dead_time_copy = dead_time_list.copy()
    if done :
        if len(dead_time_copy)>0 :
            for dead_time in dead_time_copy :
                core_id = dead_time['core']
                max_times = max(time_stamp,proc_end_times[core_id])
                dead_time['end'] = max_times
                proc_end_times[core_id] = max_times
    else :
        dead_time_copy.append({'core':ind, 'start':time_stamp, 'end':0})
    # for core in dead_time_copy :
    #     dead_time_copy[core]['duration'] = dead_time_copy[core]['end'] - dead_time_copy[core]['start']
    return dead_time_copy

def fill_schedule():
    dead_time_list = []
    end_times = get_end_times(schedule)
    # print('end_times:', end_times)
    done = False
    # print('prio',priority_list)
    task = get_next_task()
    # print('next task in prio')
    while not done :
        ind, time_stamp, end_times = gef_first_core_ready(end_times)
        # print('available core :', ind)
        # print('available at', time_stamp)
        # print('end times:', end_times)
        # print('ready_to_start', ready_to_start)
        # print(ind, time_stamp)
        if len(end_times)==0 :
            done = try_assign_process(task, ind, time_stamp, -1)
        else :
            next_end_time = max(end_times.values())
            if next_end_time == time_stamp :
                done = try_assign_process(task, ind, time_stamp, -1)
            else :
                done = try_assign_process(task, ind, time_stamp, next_end_time)

        # print('done?', done)
        dead_time_list = update_dead_time(dead_time_list, done, time_stamp, ind)
    return dead_time_list

def find_dead_time(dead_time_list) :
    max_val = -1
    max_ind = -1
    for i in range(len(dead_time_list)):
        new_val = dead_time_list[i]['start']
        if new_val>max_val :
            max_val = new_val
            max_ind = i
    # idle_intervals = []
    # for interval in dead_time_copy :
    #     idle_intervals.append(interval['start'])
    # i = np.argmax(idle_intervals)
    return max_ind


def fill_dead_time(dead_time_list):
    dead_time_copy = dead_time_list.copy()
    
    while len(dead_time_copy) > 0 :
        # print('dead time:',dead_time_copy)
        i = find_dead_time(dead_time_copy)
        dead_time_end = dead_time_copy[i]['end']
        dead_time_start = dead_time_copy[i]['start']
        idle_core = dead_time_copy[i]['core']
        possible_tasks = {}

        for task in ready_to_start :
            task_available_start = ready_to_start[task]
            start_time = max(dead_time_start,task_available_start)
            if task_available_start<=dead_time_end and nodes[task]['Weight']<= dead_time_end - start_time :
                possible_tasks[task]=nodes[task]['rank']
        
        if len(possible_tasks)>0 :
            # print('possible tasks to fill dead time', possible_tasks)
            
            task_to_do = max(possible_tasks, key=possible_tasks.get)
            # print(task_to_do)
            task_start_time = ready_to_start[task_to_do]
            # print('task start time',task_start_time)
            start_time = max(dead_time_start,task_start_time)
            task_end_time = start_time + weight(nodes[task])
            done = try_assign_process(task_to_do, idle_core, start_time, dead_time_end)
            if not done :
                print('could\'nt assign the task {} to dead time'.format(task_to_do))


            if dead_time_start<task_start_time :
                new_element = {'core':idle_core, 'start':dead_time_start, 'end':task_start_time}
                dead_time_copy.append({'core':idle_core, 'start':dead_time_start, 'end':task_start_time})
                # print('new before', new_element)
            if task_end_time<dead_time_end :
                new_element = {'core':idle_core, 'start':task_end_time, 'end':dead_time_end}
                dead_time_copy.append(new_element)
                # print('new after')
        # print(i, dead_time_copy)
        dead_time_copy.pop(i)
        # print('did pop?', dead_time_copy)
   

    


In [446]:
end_times = get_end_times(schedule)
print(end_times)


{0: 0, 1: 0}


In [447]:
n_cores = 2
schedule = make_empty_schedule(n_cores)
reinitialise_finished()
proc_end_times={core:0 for core in range(n_cores)}
priority_list = priority_list_save.copy()
ready_to_start = ready_to_start_save.copy()
all_done = False
while not all_done :
    # print('ready_to_start', ready_to_start)
    all_done = True
    dead_time_list = fill_schedule()
    # print('dead_time', dead_time_list)
    fill_dead_time(dead_time_list)
    if len(ready_to_start)>0 :
        all_done = False
    else : 
        print('no task ready to start, verifying all tasks are done')  
        if len(priority_list)>0: 
            for node in nodes :
                if nodes[node]['finished']==False :
                    # print('task not finished :', node)
                    all_done = False
    # print(schedule)

    


no task ready to start, verifying all tasks are done


In [448]:
def get_score_1(schedule):
    score = 0
    for core_list in schedule.values() :
        if len(core_list)>0 :
            new_score = core_list[-1]['end']
            if new_score>score :
                score = new_score
    return score

def get_score_2(schedule):
    score = 0
    for core_list in schedule.values() :
        for task in core_list :
            new_score = task['end']
            if new_score>score :
                score = new_score
    return score


In [449]:
get_score_1(schedule)


2292921.3700811993

In [450]:
import pandas as pd

def trasnform_schedule_to_pd(schedule):
    table = []
    for core_id in schedule :
        for task in schedule[core_id] :
            rank_val = nodes[task['task']]['rank']
            val = dict(Task = task['task'], Core = core_id, Start = task['start'], End = task['end'], Rank = rank_val)
            table.append(val)
    return pd.DataFrame(table)        


In [451]:
from pathlib import Path  
pd_schedule = trasnform_schedule_to_pd(schedule)
print(pd_schedule)
path_name = 'schedules/{}_{}cores.csv'.format(names[file_num],n_cores)
filepath = Path(path_name)  
filepath.parent.mkdir(parents=True, exist_ok=True)  
pd_schedule.to_csv(filepath)  


      Task  Core         Start           End          Rank
0      919     0  0.000000e+00  3.838084e+03  1.435771e+06
1      927     0  3.838084e+03  6.392464e+03  1.431933e+06
2      930     0  6.392464e+03  8.055724e+03  1.429379e+06
3      931     0  8.055724e+03  1.141912e+04  1.427715e+06
4      933     0  1.141912e+04  1.695538e+04  1.424352e+06
...    ...   ...           ...           ...           ...
1176   322     1  2.286732e+06  2.289728e+06  2.995978e+03
1177  1364     1  2.289728e+06  2.290571e+06  2.037780e+03
1178  1365     1  2.290571e+06  2.291766e+06  1.194478e+03
1179   673     1  2.291766e+06  2.292417e+06  6.519228e+02
1180   394     1  2.292417e+06  2.292892e+06  4.746759e+02

[1181 rows x 5 columns]
